In [ ]:
pip install vmdpy

In [1]:
from pathlib import Path
import numpy as np
import pandas as pd

import mne

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

import warnings
warnings.filterwarnings("ignore")

DATA_FOLDER = "/kaggle/input/ahmadi-dataset"
INFO_CSV = f"{DATA_FOLDER}/subject-info.csv"

RESAMPLE_TO = 128
RANDOM_STATE = 42
EPOCHS = 10
BATCH_SIZE = 8
OUTPUT_EXCEL = "results_orig_perm_vsd.xlsx"

N_TEST_SUBJECTS = 10
VSD_K_CHANNELS = 25


def load_task_edf_to_tensor(folder_path, info_csv_path, resample_to=None):
    folder = Path(folder_path)
    if not folder.is_dir():
        raise NotADirectoryError(f"{folder_path} is not a valid directory")

    info_df = pd.read_csv(info_csv_path)
    label_map = dict(zip(info_df["Subject"], info_df["Count quality"]))

    X_list = []
    y_list = []
    subjects = []
    sfreq = None

    for file_path in sorted(folder.glob("Subject*_2.edf")):
        subj = file_path.stem.split("_")[0]
        if subj not in label_map:
            print(f"Warning: {subj} not in subject-info, skipping.")
            continue

        print(f"Loading {file_path.name} for {subj}...")
        raw = mne.io.read_raw_edf(file_path, preload=True, verbose=False)

        if resample_to is not None:
            raw.resample(resample_to)

        if sfreq is None:
            sfreq = raw.info["sfreq"]

        data = raw.get_data()
        X_list.append(data)
        y_list.append(int(label_map[subj]))
        subjects.append(subj)

    if not X_list:
        raise ValueError("No *_2.edf files loaded. Check folder path and file pattern.")

    lengths = [d.shape[1] for d in X_list]
    min_len = min(lengths)

    X = np.stack([d[:, :min_len] for d in X_list], axis=0)
    y = np.array(y_list, dtype=int)
    subjects = np.array(subjects)

    print("Final tensor X shape (N, C, T):", X.shape)
    print("Labels y shape:", y.shape)

    return X, y, subjects, sfreq


def build_lstm_model(timesteps, n_features):
    model = Sequential()
    model.add(LSTM(64, input_shape=(timesteps, n_features), return_sequences=False))
    model.add(Dropout(0.5))
    model.add(Dense(32, activation="relu"))
    model.add(Dense(1, activation="sigmoid"))
    model.compile(
        optimizer=Adam(learning_rate=1e-3),
        loss="binary_crossentropy",
        metrics=["accuracy"]
    )
    return model


def compute_channel_importance_with_rf(X_train_raw, y_train, random_state):
    N, C, T = X_train_raw.shape
    X_rf = X_train_raw.reshape(N, C * T)
    rf = RandomForestClassifier(
        n_estimators=300,
        max_depth=None,
        random_state=random_state,
        n_jobs=-1
    )
    rf.fit(X_rf, y_train)
    importances = rf.feature_importances_
    importances_2d = importances.reshape(C, T)
    channel_importance = importances_2d.mean(axis=1)
    chan_perm = np.argsort(-channel_importance)
    print("\nChannel importance (sorted):")
    for rank, ch in enumerate(chan_perm):
        print(f"Rank {rank+1}: Channel {ch} -> importance {channel_importance[ch]:.6f}")
    return chan_perm, channel_importance


def main():
    X, y, subjects, sfreq = load_task_edf_to_tensor(
        DATA_FOLDER,
        INFO_CSV,
        resample_to=RESAMPLE_TO
    )

    N, C, T = X.shape
    subj_list = list(subjects)
    unique_subjects = np.array(sorted(set(subj_list), key=subj_list.index))
    n_subjects = len(unique_subjects)

    max_test_subjects = (n_subjects - 1) // 2
    print(f"\nTotal subjects: {n_subjects}, max N_TEST_SUBJECTS you can use: {max_test_subjects}")
    if N_TEST_SUBJECTS < 1 or N_TEST_SUBJECTS > max_test_subjects:
        raise ValueError(f"N_TEST_SUBJECTS={N_TEST_SUBJECTS} is invalid. It must be between 1 and {max_test_subjects}.")

    k_eff = min(VSD_K_CHANNELS, C)
    print(f"VSD_K_CHANNELS requested: {VSD_K_CHANNELS}, effective K: {k_eff} (<= number of channels {C})")

    results_lstm_orig = []
    results_lstm_perm = []
    results_lstm_perm_vsd = []

    results_models_orig = {m: [] for m in ["KNN", "RandomForest", "DecisionTree", "XGBoost"]}
    results_models_perm = {m: [] for m in ["KNN", "RandomForest", "DecisionTree", "XGBoost"]}
    results_models_perm_vsd = {m: [] for m in ["KNN", "RandomForest", "DecisionTree", "XGBoost"]}

    all_rows = []

    gpus = tf.config.list_physical_devices("GPU")
    if gpus:
        device_name = "/GPU:0"
        print("\nUsing GPU")
    else:
        device_name = "/CPU:0"
        print("\nGPU not found, using CPU")

    for fold_idx in range(n_subjects):
        test_indices = [(fold_idx + i) % n_subjects for i in range(N_TEST_SUBJECTS)]
        val_indices = [(fold_idx + N_TEST_SUBJECTS + i) % n_subjects for i in range(N_TEST_SUBJECTS)]

        test_subj = unique_subjects[test_indices]
        val_subj = unique_subjects[val_indices]

        test_mask = np.isin(subjects, test_subj)
        val_mask = np.isin(subjects, val_subj)
        train_mask = ~(test_mask | val_mask)

        X_train_raw = X[train_mask]
        X_val_raw = X[val_mask]
        X_test_raw = X[test_mask]

        y_train = y[train_mask]
        y_val = y[val_mask]
        y_test = y[test_mask]

        test_subj_str = ",".join(map(str, test_subj))
        val_subj_str = ",".join(map(str, val_subj))

        print("\n" + "=" * 60)
        print(f"Fold {fold_idx+1}/{n_subjects}")
        print(f"Train subjects: {np.unique(subjects[train_mask])}")
        print(f"Val subjects  : {val_subj}")
        print(f"Test subjects : {test_subj}")

        X_train_flat_orig = X_train_raw.reshape(X_train_raw.shape[0], C * T)
        X_test_flat_orig = X_test_raw.reshape(X_test_raw.shape[0], C * T)

        models = {
            "KNN": KNeighborsClassifier(n_neighbors=3),
            "RandomForest": RandomForestClassifier(
                n_estimators=300,
                max_depth=None,
                random_state=RANDOM_STATE,
                n_jobs=-1
            ),
            "DecisionTree": DecisionTreeClassifier(
                max_depth=None,
                random_state=RANDOM_STATE
            ),
            "XGBoost": XGBClassifier(
                n_estimators=300,
                max_depth=4,
                learning_rate=0.05,
                subsample=0.8,
                colsample_bytree=0.8,
                objective="binary:logistic",
                eval_metric="logloss",
                n_jobs=-1,
                tree_method="hist"
            ),
        }

        for name, model in models.items():
            model.fit(X_train_flat_orig, y_train)
            y_pred_m = model.predict(X_test_flat_orig)
            acc_m = accuracy_score(y_test, y_pred_m)
            prec_m = precision_score(y_test, y_pred_m, zero_division=0)
            rec_m = recall_score(y_test, y_pred_m, zero_division=0)
            f1_m = f1_score(y_test, y_pred_m, zero_division=0)
            results_models_orig[name].append([acc_m, prec_m, rec_m, f1_m])
            all_rows.append({
                "fold": fold_idx + 1,
                "test_subject": test_subj_str,
                "val_subject": val_subj_str,
                "model": name,
                "type": "classic",
                "order": "original",
                "accuracy": acc_m,
                "precision": prec_m,
                "recall": rec_m,
                "f1": f1_m
            })
            print(f"{name} (original) -> acc: {acc_m:.4f}, prec: {prec_m:.4f}, rec: {rec_m:.4f}, f1: {f1_m:.4f}")

        X_train_seq_orig = np.transpose(X_train_raw, (0, 2, 1))
        X_val_seq_orig = np.transpose(X_val_raw, (0, 2, 1))
        X_test_seq_orig = np.transpose(X_test_raw, (0, 2, 1))

        timesteps = X_train_seq_orig.shape[1]
        n_features = X_train_seq_orig.shape[2]

        with tf.device(device_name):
            tf.keras.backend.clear_session()
            tf.random.set_seed(RANDOM_STATE)
            model_lstm_orig = build_lstm_model(timesteps, n_features)
            model_lstm_orig.fit(
                X_train_seq_orig,
                y_train,
                epochs=EPOCHS,
                batch_size=BATCH_SIZE,
                validation_data=(X_val_seq_orig, y_val),
                verbose=1
            )
            y_pred_prob_orig = model_lstm_orig.predict(X_test_seq_orig)

        y_pred_lstm_orig = (y_pred_prob_orig >= 0.5).astype(int).ravel()

        acc_o = accuracy_score(y_test, y_pred_lstm_orig)
        prec_o = precision_score(y_test, y_pred_lstm_orig, zero_division=0)
        rec_o = recall_score(y_test, y_pred_lstm_orig, zero_division=0)
        f1_o = f1_score(y_test, y_pred_lstm_orig, zero_division=0)

        results_lstm_orig.append([acc_o, prec_o, rec_o, f1_o])
        all_rows.append({
            "fold": fold_idx + 1,
            "test_subject": test_subj_str,
            "val_subject": val_subj_str,
            "model": "LSTM",
            "type": "lstm",
            "order": "original",
            "accuracy": acc_o,
            "precision": prec_o,
            "recall": rec_o,
            "f1": f1_o
        })
        print(f"LSTM (original) -> acc: {acc_o:.4f}, prec: {prec_o:.4f}, rec: {rec_o:.4f}, f1: {f1_o:.4f}")

        chan_perm, chan_scores = compute_channel_importance_with_rf(
            X_train_raw, y_train, RANDOM_STATE
        )

        X_train_raw_perm = X_train_raw[:, chan_perm, :]
        X_val_raw_perm = X_val_raw[:, chan_perm, :]
        X_test_raw_perm = X_test_raw[:, chan_perm, :]

        X_train_flat_perm = X_train_raw_perm.reshape(X_train_raw_perm.shape[0], C * T)
        X_test_flat_perm = X_test_raw_perm.reshape(X_test_raw_perm.shape[0], C * T)

        for name, model in models.items():
            model.fit(X_train_flat_perm, y_train)
            y_pred_m = model.predict(X_test_flat_perm)
            acc_m = accuracy_score(y_test, y_pred_m)
            prec_m = precision_score(y_test, y_pred_m, zero_division=0)
            rec_m = recall_score(y_test, y_pred_m, zero_division=0)
            f1_m = f1_score(y_test, y_pred_m, zero_division=0)
            results_models_perm[name].append([acc_m, prec_m, rec_m, f1_m])
            all_rows.append({
                "fold": fold_idx + 1,
                "test_subject": test_subj_str,
                "val_subject": val_subj_str,
                "model": name,
                "type": "classic",
                "order": "perm",
                "accuracy": acc_m,
                "precision": prec_m,
                "recall": rec_m,
                "f1": f1_m
            })
            print(f"{name} (permuted) -> acc: {acc_m:.4f}, prec: {prec_m:.4f}, rec: {rec_m:.4f}, f1: {f1_m:.4f}")

        X_train_seq_perm = np.transpose(X_train_raw_perm, (0, 2, 1))
        X_val_seq_perm = np.transpose(X_val_raw_perm, (0, 2, 1))
        X_test_seq_perm = np.transpose(X_test_raw_perm, (0, 2, 1))

        timesteps_p = X_train_seq_perm.shape[1]
        n_features_p = X_train_seq_perm.shape[2]

        with tf.device(device_name):
            tf.keras.backend.clear_session()
            tf.random.set_seed(RANDOM_STATE)
            model_lstm_perm = build_lstm_model(timesteps_p, n_features_p)
            model_lstm_perm.fit(
                X_train_seq_perm,
                y_train,
                epochs=EPOCHS,
                batch_size=BATCH_SIZE,
                validation_data=(X_val_seq_perm, y_val),
                verbose=1
            )
            y_pred_prob_perm = model_lstm_perm.predict(X_test_seq_perm)

        y_pred_lstm_perm = (y_pred_prob_perm >= 0.5).astype(int).ravel()

        acc_p = accuracy_score(y_test, y_pred_lstm_perm)
        prec_p = precision_score(y_test, y_pred_lstm_perm, zero_division=0)
        rec_p = recall_score(y_test, y_pred_lstm_perm, zero_division=0)
        f1_p = f1_score(y_test, y_pred_lstm_perm, zero_division=0)

        results_lstm_perm.append([acc_p, prec_p, rec_p, f1_p])
        all_rows.append({
            "fold": fold_idx + 1,
            "test_subject": test_subj_str,
            "val_subject": val_subj_str,
            "model": "LSTM",
            "type": "lstm",
            "order": "perm",
            "accuracy": acc_p,
            "precision": prec_p,
            "recall": rec_p,
            "f1": f1_p
        })
        print(f"LSTM (permuted) -> acc: {acc_p:.4f}, prec: {prec_p:.4f}, rec: {rec_p:.4f}, f1: {f1_p:.4f}")

        X_train_raw_vsd = X_train_raw_perm[:, :k_eff, :]
        X_val_raw_vsd = X_val_raw_perm[:, :k_eff, :]
        X_test_raw_vsd = X_test_raw_perm[:, :k_eff, :]

        X_train_flat_vsd = X_train_raw_vsd.reshape(X_train_raw_vsd.shape[0], k_eff * T)
        X_test_flat_vsd = X_test_raw_vsd.reshape(X_test_raw_vsd.shape[0], k_eff * T)

        for name, model in models.items():
            model.fit(X_train_flat_vsd, y_train)
            y_pred_m = model.predict(X_test_flat_vsd)
            acc_m = accuracy_score(y_test, y_pred_m)
            prec_m = precision_score(y_test, y_pred_m, zero_division=0)
            rec_m = recall_score(y_test, y_pred_m, zero_division=0)
            f1_m = f1_score(y_test, y_pred_m, zero_division=0)
            results_models_perm_vsd[name].append([acc_m, prec_m, rec_m, f1_m])
            all_rows.append({
                "fold": fold_idx + 1,
                "test_subject": test_subj_str,
                "val_subject": val_subj_str,
                "model": name,
                "type": "classic",
                "order": "perm_vsd",
                "accuracy": acc_m,
                "precision": prec_m,
                "recall": rec_m,
                "f1": f1_m
            })
            print(f"{name} (perm_vsd, K={k_eff}) -> acc: {acc_m:.4f}, prec: {prec_m:.4f}, rec: {rec_m:.4f}, f1: {f1_m:.4f}")

        X_train_seq_vsd = np.transpose(X_train_raw_vsd, (0, 2, 1))
        X_val_seq_vsd = np.transpose(X_val_raw_vsd, (0, 2, 1))
        X_test_seq_vsd = np.transpose(X_test_raw_vsd, (0, 2, 1))

        timesteps_v = X_train_seq_vsd.shape[1]
        n_features_v = X_train_seq_vsd.shape[2]

        with tf.device(device_name):
            tf.keras.backend.clear_session()
            tf.random.set_seed(RANDOM_STATE)
            model_lstm_vsd = build_lstm_model(timesteps_v, n_features_v)
            model_lstm_vsd.fit(
                X_train_seq_vsd,
                y_train,
                epochs=EPOCHS,
                batch_size=BATCH_SIZE,
                validation_data=(X_val_seq_vsd, y_val),
                verbose=1
            )
            y_pred_prob_vsd = model_lstm_vsd.predict(X_test_seq_vsd)

        y_pred_lstm_vsd = (y_pred_prob_vsd >= 0.5).astype(int).ravel()

        acc_v = accuracy_score(y_test, y_pred_lstm_vsd)
        prec_v = precision_score(y_test, y_pred_lstm_vsd, zero_division=0)
        rec_v = recall_score(y_test, y_pred_lstm_vsd, zero_division=0)
        f1_v = f1_score(y_test, y_pred_lstm_vsd, zero_division=0)

        results_lstm_perm_vsd.append([acc_v, prec_v, rec_v, f1_v])
        all_rows.append({
            "fold": fold_idx + 1,
            "test_subject": test_subj_str,
            "val_subject": val_subj_str,
            "model": "LSTM",
            "type": "lstm",
            "order": "perm_vsd",
            "accuracy": acc_v,
            "precision": prec_v,
            "recall": rec_v,
            "f1": f1_v
        })
        print(f"LSTM (perm_vsd, K={k_eff}) -> acc: {acc_v:.4f}, prec: {prec_v:.4f}, rec: {rec_v:.4f}, f1: {f1_v:.4f}")

    results_lstm_orig = np.array(results_lstm_orig)
    results_lstm_perm = np.array(results_lstm_perm)
    results_lstm_perm_vsd = np.array(results_lstm_perm_vsd)

    print("\n" + "=" * 60)
    print("Average metrics over subject-wise folds:")

    print("\nLSTM (original):")
    print(f"Accuracy : {results_lstm_orig[:,0].mean():.4f}")
    print(f"Precision: {results_lstm_orig[:,1].mean():.4f}")
    print(f"Recall   : {results_lstm_orig[:,2].mean():.4f}")
    print(f"F1-score : {results_lstm_orig[:,3].mean():.4f}")

    print("\nLSTM (perm):")
    print(f"Accuracy : {results_lstm_perm[:,0].mean():.4f}")
    print(f"Precision: {results_lstm_perm[:,1].mean():.4f}")
    print(f"Recall   : {results_lstm_perm[:,2].mean():.4f}")
    print(f"F1-score : {results_lstm_perm[:,3].mean():.4f}")

    print("\nLSTM (perm_vsd):")
    print(f"Accuracy : {results_lstm_perm_vsd[:,0].mean():.4f}")
    print(f"Precision: {results_lstm_perm_vsd[:,1].mean():.4f}")
    print(f"Recall   : {results_lstm_perm_vsd[:,2].mean():.4f}")
    print(f"F1-score : {results_lstm_perm_vsd[:,3].mean():.4f}")

    for name in results_models_orig.keys():
        arr_o = np.array(results_models_orig[name])
        arr_p = np.array(results_models_perm[name])
        arr_v = np.array(results_models_perm_vsd[name])
        print(f"\n{name} (original):")
        print(f"Accuracy : {arr_o[:,0].mean():.4f}")
        print(f"Precision: {arr_o[:,1].mean():.4f}")
        print(f"Recall   : {arr_o[:,2].mean():.4f}")
        print(f"F1-score : {arr_o[:,3].mean():.4f}")
        print(f"{name} (perm):")
        print(f"Accuracy : {arr_p[:,0].mean():.4f}")
        print(f"Precision: {arr_p[:,1].mean():.4f}")
        print(f"Recall   : {arr_p[:,2].mean():.4f}")
        print(f"F1-score : {arr_p[:,3].mean():.4f}")
        print(f"{name} (perm_vsd, K={k_eff}):")
        print(f"Accuracy : {arr_v[:,0].mean():.4f}")
        print(f"Precision: {arr_v[:,1].mean():.4f}")
        print(f"Recall   : {arr_v[:,2].mean():.4f}")
        print(f"F1-score : {arr_v[:,3].mean():.4f}")

    results_df = pd.DataFrame(all_rows)
    results_df.to_excel(OUTPUT_EXCEL, index=False)
    print("\nAll detailed results saved to:", OUTPUT_EXCEL)


if __name__ == "__main__":
    main()


2025-12-11 07:39:02.232575: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765438742.431677      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765438742.489284      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

RandomForest (original) -> acc: 0.7000, prec: 0.7000, rec: 1.0000, f1: 0.8235
DecisionTree (original) -> acc: 0.4000, prec: 0.5714, rec: 0.5714, f1: 0.5714
XGBoost (original) -> acc: 0.7000, prec: 0.7000, rec: 1.0000, f1: 0.8235
Epoch 1/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 581ms/step - accuracy: 0.5417 - loss: 0.6923 - val_accuracy: 0.7000 - val_loss: 0.6900
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 358ms/step - accuracy: 0.7083 - loss: 0.6889 - val_accuracy: 0.7000 - val_loss: 0.6862
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 346ms/step - accuracy: 0.7083 - loss: 0.6839 - val_accuracy: 0.7000 - val_loss: 0.6811
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 359ms/step - accuracy: 0.7083 - loss: 0.6769 - val_accuracy: 0.7000 - val_loss: 0.6746
Epoch 5/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 347ms/step - accuracy: 0.7083 - loss: 0.6739 - val_accuracy: 0.7000 - val_loss: 0.6661
Epoch 6/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 345ms/step - accuracy: 0.7083 - loss: 0.6661 - val_accuracy: 0.7000 - val_loss: 0.6547
Epoch 7/10
2/2 

OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detec

RandomForest (permuted) -> acc: 0.7000, prec: 0.7000, rec: 1.0000, f1: 0.8235
DecisionTree (permuted) -> acc: 0.6000, prec: 0.8000, rec: 0.5714, f1: 0.6667
XGBoost (permuted) -> acc: 0.7000, prec: 0.7000, rec: 1.0000, f1: 0.8235
Epoch 1/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 621ms/step - accuracy: 0.5417 - loss: 0.6921 - val_accuracy: 0.7000 - val_loss: 0.6895
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 344ms/step - accuracy: 0.7083 - loss: 0.6878 - val_accuracy: 0.7000 - val_loss: 0.6846
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 346ms/step - accuracy: 0.7083 - loss: 0.6817 - val_accuracy: 0.7000 - val_loss: 0.6789
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 343ms/step - accuracy: 0.7083 - loss: 0.6778 - val_accuracy: 0.7000 - val_loss: 0.6717
Epoch 5/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 343ms/step - accuracy: 0.7083 - loss: 0.6710 - val_accuracy: 0.7000 - val_loss: 0.6626
Epoch 6/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 345ms/step - accuracy: 0.7083 - loss: 0.6549 - val_accuracy: 0.7000 - val_loss: 0.6495
Epoch 7/10
2/2 

OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detec

RandomForest (perm_vsd, K=10) -> acc: 0.7000, prec: 0.7000, rec: 1.0000, f1: 0.8235
DecisionTree (perm_vsd, K=10) -> acc: 0.8000, prec: 1.0000, rec: 0.7143, f1: 0.8333
XGBoost (perm_vsd, K=10) -> acc: 0.7000, prec: 0.7000, rec: 1.0000, f1: 0.8235
Epoch 1/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 590ms/step - accuracy: 0.5417 - loss: 0.6924 - val_accuracy: 0.7000 - val_loss: 0.6902
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 342ms/step - accuracy: 0.7083 - loss: 0.6891 - val_accuracy: 0.7000 - val_loss: 0.6864
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 345ms/step - accuracy: 0.7083 - loss: 0.6854 - val_accuracy: 0.7000 - val_loss: 0.6820
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 342ms/step - accuracy: 0.7083 - loss: 0.6814 - val_accuracy: 0.7000 - val_loss: 0.6769
Epoch 5/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 345ms/step - accuracy: 0.7083 - loss: 0.6751 - val_accuracy: 0.7000 - val_loss: 0.6703
Epoch 6/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 342ms/step - accuracy: 0.7083 - loss: 0.6677 - val_accuracy: 0.7000 - val_loss: 0.66